# training set : generation 1

In [1]:
from ase import Atoms,Atom
from ase.io import read,write
from ase.build import make_supercell,sort

import numpy as np

## 0. **structure optimize**

by DFT calculation get CONTCAR at this time, CONTCAR is Ground state

## 1. **Distortion**

* The structure obtained through structure optimization is used 

* cell It distorts the size of the cell and the position of the ion

In [2]:
max_displacement=0.01 # Maximum displacement in angstrom
max_cell_change=0.01 # Maximum fractional change in cell

In [3]:
bulk_ruo2 = read('CONTCAR')
P = np.identity(3) * 2
conf = make_supercell(bulk_ruo2, P)

initial_positions=conf.get_positions()
initial_cell=conf.get_cell()

In [4]:
num_iterations= 1
for i in range(num_iterations):
    positions=np.copy(initial_positions)
    cell=np.copy(initial_cell)
    # Displace each coordinate randomly
    positions += np.random.rand(positions.shape[0],positions.shape[1])*2*max_displacement - max_displacement
    conf.set_positions(positions)
    # Scale each cell component randomly
    cell *= 1-(np.random.rand(cell.shape[0],cell.shape[1])*2*max_cell_change-max_cell_change)
    conf.set_cell(cell,scale_atoms=True)
    write('POSCAR-' + str(i),sort(conf))

## 2.  **Oxygen vacancy**

* The structure obtained through structure optimization is used

* Randomly add a vacancy using atom.index

In [5]:
bulk_ruo2
P = np.identity(3) * 2
conff = make_supercell(bulk_ruo2, P)

In [6]:
count_oxygen = sum(conf.symbols == 'O')
count_ru = len(conf) - count_oxygen
def vacancy(structure,ratio):
    count_oxygen = sum(structure.symbols == 'O')
    count_ru = len(structure) - count_oxygen
    x = - (ratio * count_ru - count_oxygen)
    xx = np.random.choice(range(0,count_oxygen - 1), int(x),replace=False)
    xx.sort()
    conf = sort(structure)
    for i in xx:
        del conf[int(i) - i]
    return conf
        

In [7]:
ruo15 = vacancy(conff,1.5)
ruo175 = vacancy(conff,1.75)

In [8]:
write('ruo15.xyz',sort(ruo15))
write('ruo175.xyz',sort(ruo175))